# Recurrent Neural Network (RNN)
In the following we will implement and train an RNN for text classification. The following example will use the ATIS database. The ATIS database consists of data obtained from the Official Airline Guide (OAG, 1990), organized under a relational schema. The database remained fixed throughout the pilot phase. It contains information about flights, fares, airlines, cities, airports, and ground services, and includes twenty-five supporting tables. The large majority of the questions posed by subjects can be answered from the database with a single relational query.

For more information about RNN check out the following resources:

[RNN with Theano](http://deeplearning.net/tutorial/rnnslu.html)

[Nice blog about RNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

[Amazing Stanford lecture about NLP (natural language processing)][http://cs224d.stanford.edu/]

For advanced text processing:

[Long-Short-Term-Memory (LSTM) a RNN variant](http://deeplearning.net/tutorial/lstm.html)

In [1]:
import theano, numpy
from theano import tensor as T
import os
import pickle
import copy
import sys
import gzip
import urllib
import random
import stat
import subprocess
import sys
import timeit
from sklearn.neighbors import NearestNeighbors
import numpy as np
from collections import OrderedDict
from sklearn.neighbors import NearestNeighbors
import numpy as np
sys.setrecursionlimit(1500)

# get home directory
from os.path import expanduser
PREFIX = expanduser("~")

## Helper functions

In [2]:
def download(origin, destination):
    '''
    download the corresponding atis file
    from http://www-etud.iro.umontreal.ca/~mesnilgr/atis/
    '''
    print ('Downloading data from %s' % origin)
    urllib.urlretrieve(origin, destination)


# data loading functions
def atisfold(fold):
    assert fold in range(5)
    filename = os.path.join(PREFIX, 'ml-bootcamp/atis/atis.fold'+str(fold)+'.pkl.gz')
    print(filename)
    f = gzip.open(filename, 'rb')
    train_set, valid_set, test_set, dicts = pickle.load(f,encoding='latin1')
    return train_set, valid_set, test_set, dicts
def shuffle(lol, seed):
    '''
    lol :: list of list as input
    seed :: seed the shuffling

    shuffle inplace each list in the same order
    '''
    for l in lol:
        random.seed(seed)
        random.shuffle(l)

def contextwin(l, win):
    '''
    win :: int corresponding to the size of the window
    given a list of indexes composing a sentence

    l :: array containing the word indexes

    it will return a list of list of indexes corresponding
    to context windows surrounding each word in the sentence
    '''
    assert (win % 2) == 1
    assert win >= 1
    l = list(l)

    lpadded = win // 2 * [-1] + l + win // 2 * [-1]
    out = [lpadded[i:(i + win)] for i in range(len(l))]

    assert len(out) == len(l)
    return out
def conlleval(p, g, w, filename):
    '''
    INPUT:
    p :: predictions
    g :: groundtruth
    w :: corresponding words
    OUTPUT:
    filename :: name of the file where the predictions
    are written. it will be the input of conlleval.pl script
    for computing the performance in terms of precision
    recall and f1 score
    '''
    out = ''
    for sl, sp, sw in zip(g, p, w):
        out += 'BOS O O\n'
        for wl, wp, w in zip(sl, sp, sw):
            out += w + ' ' + wl + ' ' + wp + '\n'
        out += 'EOS O O\n\n'

    f = open(filename,'w')
    f.writelines(out)
    f.close()
    
    return get_perf(filename)

def get_perf(filename):
    ''' run conlleval.pl perl script to obtain
    precision/recall and F1 score '''
    _conlleval = os.path.join(PREFIX, 'ml-bootcamp/conlleval.pl')
    if not os.path.isfile(_conlleval):
        #download('http://www-etud.iro.umontreal.ca/~mesnilgr/atis/conlleval.pl') 
        os.system('wget https://www.comp.nus.edu.sg/%7Ekanmy/courses/practicalNLP_2008/packages/conlleval.pl')
        os.chmod('conlleval.pl', stat.S_IRWXU) # give the execute permissions

    proc = subprocess.Popen(["perl", _conlleval], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    stdout, _ = proc.communicate(open(filename).read().encode('utf-8'))

    for line in stdout.decode().split('\n'):
        if 'accuracy' in line:
            out = line.split()
            break
    
    # out = ['accuracy:', '16.26%;', 'precision:', '0.00%;', 'recall:', '0.00%;', 'FB1:', '0.00']
    
    precision = float(out[3][:-2])
    recall    = float(out[5][:-2])
    f1score   = float(out[7])

    return {'p':precision, 'r':recall, 'f1':f1score}

def get_perfo(filename):
    ''' 
    work around for using a PERL script in python
    dirty but still works.
    '''
    tempfile = str(random.randint(1,numpy.iinfo('i').max)) + '.txt'
    if not isfile(PREFIX + 'conlleval.pl'):
        os.system('wget https://www.comp.nus.edu.sg/%7Ekanmy/courses/practicalNLP_2008/packages/conlleval.pl')
        #download('http://www-etud.iro.umontreal.ca/~mesnilgr/atis/conlleval.pl') 
        chmod('conlleval.pl', stat.S_IRWXU) # give the execute permissions
    if len(PREFIX) > 0:
        chmod(PREFIX + 'conlleval.pl', stat.S_IRWXU) # give the execute permissions
        cmd = PREFIX + 'conlleval.pl < %s | grep accuracy > %s'%(filename,tempfile)
    else:
        cmd = './conlleval.pl < %s | grep accuracy > %s'%(filename,tempfile)
    print (cmd)
    out = os.system(cmd)
    out = open(tempfile).readlines()[0].split()
    os.system('rm %s'%tempfile)
    precision = float(out[6][:-2])
    recall    = float(out[8][:-2])
    f1score   = float(out[10])
    return {'p':precision, 'r':recall, 'f1':f1score}

def arraysentence2string(arr):
    return " ".join([idx2word[x] for x in arr if x > 0])

def arraylabels2string(arr):
    return " ".join([idx2label[x] for x in arr if x > 0])

# Load the data

In [3]:
# load the dataset
train_set, valid_set, test_set, dic = atisfold(0)
import six

idx2label = dict((k, v) for v, k in six.iteritems(dic['labels2idx']))
idx2word = dict((k, v) for v, k in six.iteritems(dic['words2idx']))

train_lex, train_ne, train_y = train_set
valid_lex, valid_ne, valid_y = valid_set
test_lex, test_ne, test_y = test_set

vocsize = len(dic['words2idx'])
print ("Dictionary size: "+str(vocsize))
nclasses = len(dic['labels2idx'])
print ("Number of class labels: "+str(nclasses))
nsentences = len(train_lex)

groundtruth_valid = list(list(map(lambda x: idx2label[x], y)) for y in valid_y)
words_valid = list(list(map(lambda x: idx2word[x], w)) for w in valid_lex)
groundtruth_test = list(list(map(lambda x: idx2label[x], y)) for y in test_y)
words_test = list(list(map(lambda x: idx2word[x], w)) for w in test_lex)

/home/i311008/ml-bootcamp/atis/atis.fold0.pkl.gz
Dictionary size: 572
Number of class labels: 127


In [30]:
print (train_lex)

[array([232,   0, 273, 502, 254, 481, 437, 515,  73,  77,  40, 415, 205], dtype=int32), array([554, 194, 208, 379, 502, 333, 358, 317], dtype=int32), array([232, 565, 273,  13, 193, 208, 376, 502, 128, 358,  37,  26], dtype=int32), array([194, 208, 229, 502, 285,  41, 139,  19,  11], dtype=int32), array([554, 194,  50, 484, 208, 471, 502,  97], dtype=int32), array([194, 208, 328, 502, 423], dtype=int32), array([554, 526, 353,  23, 157, 167, 196, 208,  64, 502, 137,  72,   8, 384], dtype=int32), array([241, 484,  13, 193, 358, 120,  26, 208,  77, 502, 137], dtype=int32), array([232, 565, 273, 502, 254, 481, 190, 105, 182, 358,  13, 193, 208,
        69, 502, 137], dtype=int32), array([212, 301, 194, 208, 137, 502,  69], dtype=int32), array([554, 241, 481, 121, 353,  13, 498, 215, 208, 137, 502,  77, 248,
       524, 188,  11], dtype=int32), array([230, 324, 241,  13, 193, 208, 543, 502,  77], dtype=int32), array([232, 331,  13, 277, 353, 194, 208, 452, 375, 195, 502, 466, 543,
       27

In [4]:
words_valid

[['what',
  'aircraft',
  'is',
  'used',
  'on',
  'delta',
  'flight',
  'DIGITDIGITDIGITDIGIT',
  'from',
  'kansas',
  'city',
  'to',
  'salt',
  'lake',
  'city'],
 ['i', 'want', 'to', 'go', 'from', 'boston', 'to', 'atlanta', 'on', 'monday'],
 ['i',
  'need',
  'a',
  'flight',
  'from',
  'atlanta',
  'to',
  'philadelphia',
  'and',
  'i',
  "'m",
  'looking',
  'for',
  'the',
  'cheapest',
  'fare'],
 ['i',
  'need',
  'a',
  'flight',
  'from',
  'toronto',
  'to',
  'montreal',
  'reaching',
  'montreal',
  'early',
  'on',
  'friday'],
 ['show',
  'me',
  'the',
  'evening',
  'flights',
  'from',
  'philadelphia',
  'to',
  'baltimore'],
 ['tell', 'me', 'distance', 'from', 'orlando', 'airport', 'to', 'the', 'city'],
 ['what', 'is', 'restriction', 'ap80'],
 ['what',
  'is',
  'the',
  'lowest',
  'cost',
  'fare',
  'that',
  'delta',
  'has',
  'between',
  'boston',
  'and',
  'san',
  'francisco'],
 ['flight', 'DIGITDIGITDIGIT', 'from', 'cincinnati', 'to', 'dallas'],
 [

# How does the data look like?
In order to represent sentences and to make them machine understandable, data has to be processed. At first a dictionary is built. Within the dictionary each word is associated with an index. Given the dictionary and a sentence, a normal textural sentence is converted an array of indices.

In [5]:
# The first sentence in the training data set
train_lex[0]

array([554, 194, 268,  64,  62,  16,   8, 234, 481,  20,  40,  58, 234,
       415, 205], dtype=int32)

In [6]:
# The first sentence in the training data set
example_sentence = train_lex[0]
print(example_sentence)

[554 194 268  64  62  16   8 234 481  20  40  58 234 415 205]


In [7]:
print(arraysentence2string(example_sentence))

what flights leave atlanta at about DIGIT in the afternoon and arrive in san francisco


In [8]:
# The first label in the training data set
example_labels = train_y[0]
print(example_labels)

[126 126 126  48 126  36  35 126 126  33 126 126 126  78 123]


In [9]:
print(arraylabels2string(example_labels))

O O O B-fromloc.city_name O B-depart_time.time_relative B-depart_time.time O O B-depart_time.period_of_day O O O B-toloc.city_name I-toloc.city_name


## Context window
Words in itself can be ambigious, therefore it is useful to consider them in their context. As an example the word 'Paris' could relate to the actress 'Paris Hilton' or the city of Paris. Therefore each word is normally considered within its context. That is we look what comes before and after the word we are interested. Special cases are the beginning and the end of a sentence. In that case we use padding (-1) to highlight this.

In [10]:
win_size = 5 
csample = contextwin(example_sentence, win_size)

for x in csample:
        print(x)
        print(arraysentence2string(x)+'\n')

[-1, -1, 554, 194, 268]
what flights leave

[-1, 554, 194, 268, 64]
what flights leave atlanta

[554, 194, 268, 64, 62]
what flights leave atlanta at

[194, 268, 64, 62, 16]
flights leave atlanta at about

[268, 64, 62, 16, 8]
leave atlanta at about DIGIT

[64, 62, 16, 8, 234]
atlanta at about DIGIT in

[62, 16, 8, 234, 481]
at about DIGIT in the

[16, 8, 234, 481, 20]
about DIGIT in the afternoon

[8, 234, 481, 20, 40]
DIGIT in the afternoon and

[234, 481, 20, 40, 58]
in the afternoon and arrive

[481, 20, 40, 58, 234]
the afternoon and arrive in

[20, 40, 58, 234, 415]
afternoon and arrive in san

[40, 58, 234, 415, 205]
and arrive in san francisco

[58, 234, 415, 205, -1]
arrive in san francisco

[234, 415, 205, -1, -1]
in san francisco



In [37]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [12]:
# Now let's define a RNN
class RNNSLU(object):
    ''' elman neural net model '''
    def __init__(self, nh, nc, ne, de, cs):
        '''
        nh :: dimension of the hidden layer
        nc :: number of classes
        ne :: number of word embeddings in the vocabulary / size of vocabulary
        de :: dimension of the word embeddings
        cs :: word window context size
        '''
        
        # parameters of the model
        self.emb = theano.shared(name='embeddings',
                                 value=0.2 * numpy.random.uniform(-1.0, 1.0,
                                 (ne+1, de))
                                 # add one for padding at the end
                                 .astype(theano.config.floatX))
        self.wx = theano.shared(name='wx',
                                value=0.2 * numpy.random.uniform(-1.0, 1.0,
                                (de * cs, nh))
                                .astype(theano.config.floatX))
        self.wh = theano.shared(name='wh',
                                value=0.2 * numpy.random.uniform(-1.0, 1.0,
                                (nh, nh))
                                .astype(theano.config.floatX))
        self.w = theano.shared(name='w',
                               value=0.2 * numpy.random.uniform(-1.0, 1.0,
                               (nh, nc))
                               .astype(theano.config.floatX))
        self.bh = theano.shared(name='bh',
                                value=numpy.zeros(nh,
                                dtype=theano.config.floatX))
        self.b = theano.shared(name='b',
                               value=numpy.zeros(nc,
                               dtype=theano.config.floatX))
        self.h0 = theano.shared(name='h0',
                                value=numpy.zeros(nh,
                                dtype=theano.config.floatX))
        
        

        # bundle the parameters together, nice for gradient computation
        self.params = [self.emb, self.wx, self.wh, self.w,
                       self.bh, self.b, self.h0]
        
        # we perform training on a complete sentence, which is broken down into context window elements
        
        # input is a matrix full of indices:
        # as many columns as context window size
        # as many lines as words in the sentence (each word is presented within context window)
        idxs = T.imatrix()
        
        # map the input to the embedding space
        x = self.emb[idxs].reshape((idxs.shape[0], de*cs))
        
        # labels for the words in the stenence
        y_sentence = T.ivector('y_sentence')
        
        
        # basic Elmann recurrent networks
        
        def recurrence(x_t, h_tm1):
            
            # hidden state
            h_t = T.nnet.sigmoid(T.dot(x_t, self.wx)
                                 + T.dot(h_tm1, self.wh) + self.bh)
            
            # output
            s_t = T.nnet.softmax(T.dot(h_t, self.w) + self.b)
            
            return [h_t, s_t]

        # looping over the words in the sentence
        [h, s], _ = theano.scan(fn=recurrence, #first argument is the time sliced input
                                # input
                                sequences=x,
                                # initial value, must have the same dimension as output [h_t, s_t]
                                outputs_info=[self.h0, None],
                                # number of steps in the loop (corresponds to words in the sentence)
                                n_steps=x.shape[0])

        # output of the RNN is softmax result (probabilities)
        p_y_given_x_sentence = s[:, 0, :]
        
        # for each word return the class with maximum probability
        y_pred = T.argmax(p_y_given_x_sentence, axis=1)
        

        # cost and gradients and learning rate
        lr = T.scalar('lr')

        # cost function is the negative log-likelihood (NLL)
        
        
        # x.shape[0] is (symbolically) the number of rows in x, i.e.,
        # number of words in the sentence (call it n) in the minibatch
        # T.arange(x.shape[0]) is a symbolic vector which will contain
        # [0,1,2,... n-1] T.log(self.p_y_given_x_sentence) is a matrix of
        # Log-Probabilities (call it LP) with one row per word and
        # one column per class LP[T.arange(x.shape[0]),y_sentence] is a vector
        # v containing [LP[0,y_sentence[0]], LP[1,y_sentence[1]], LP[2,y_sentence[2]], ...,
        # LP[n-1,y_sentence[n-1]]] and T.mean(LP[T.arange(x.shape[0]),y_sentence]) is
        # the mean (across minibatch examples) of the elements in v,
        # i.e., the mean log-likelihood across the minibatch.
        
        sentence_nll = -T.mean(T.log(p_y_given_x_sentence)
                               [T.arange(x.shape[0]), y_sentence])
        
        # now let theano do the ugly work and compute the SYMBOLIC gradients
        sentence_gradients = T.grad(sentence_nll, self.params)
        
        # the updates is an ordered dicitionary, where each pair is a parameter and how it is updated (gradient descent)
        sentence_updates = OrderedDict((p, p - lr*g)
                                       for p, g in
                                       zip(self.params, sentence_gradients))
    

        # theano functions to compile
        
        # classification function: 
        # INPUT: sentence (put in context windows)
        # RETURN: vector (class for each word)
        self.classify = theano.function(inputs=[idxs], outputs=y_pred)
        
        # training function:
        # INPUT: sentence (put in context windows), class labels for each word (vector), and a learning rate
        # RETURN: cost (negative log likelihood)
        self.sentence_train = theano.function(inputs=[idxs, y_sentence, lr],
                                              outputs=sentence_nll, 
                                              updates=sentence_updates)
        
        # normalization function (normalization of the embedding)
        # update the embedding, such that it stays on the unit-ball (L2 norm equal to 1)
        # dimshuffle(0, 'x') ->  make a column out of a 1d vector (N to Nx1)
        self.normalize = theano.function(inputs=[],
                                         updates={self.emb:
                                                  self.emb /
                                                  T.sqrt((self.emb**2)
                                                  .sum(axis=1))
                                                  .dimshuffle(0, 'x')})


    def train(self, x, y, window_size, learning_rate):

        cwords = contextwin(x, window_size)
        #words = map(lambda x: numpy.asarray(x).astype('int32'), cwords)
        words = cwords
        labels = y
        
        #print(words,flush=True)

        self.sentence_train(words, labels, learning_rate)
        self.normalize()

    def save(self, folder):
        for param in self.params:
            numpy.save(os.path.join(folder,
                       param.name + '.npy'), param.get_value())

    def load(self, folder):
        for param in self.params:
            param.set_value(numpy.load(os.path.join(folder,
                            param.name + '.npy')))

In [35]:
# RNN (training) parameters

In [13]:
param = {
    'fold': 3,
    # 5 folds 0,1,2,3,4
    'data': 'atis',
    'lr': 0.0970806646812754,
    'verbose': 1,
    'decay': True,
    # decay on the learning rate if improvement stops
    'win': 5,
    # number of words in the context window
    'nhidden': 50,
    # number of hidden units
    'seed': 345,
    'emb_dimension': 3,
    # dimension of word embedding
    'nepochs': 30,
    # 60 is recommended
    'savemodel': False}
print(param)


{'emb_dimension': 3, 'win': 5, 'data': 'atis', 'fold': 3, 'nepochs': 30, 'nhidden': 50, 'savemodel': False, 'verbose': 1, 'seed': 345, 'decay': True, 'lr': 0.0970806646812754}


In [14]:
folder_name = "RNN"
folder = os.path.join(PREFIX, folder_name)
if not os.path.exists(folder):
    os.mkdir(folder)



# instanciate the model
numpy.random.seed(param['seed'])
random.seed(param['seed'])

rnn = RNNSLU(nh=param['nhidden'],
             nc=nclasses,
             ne=vocsize,
             de=param['emb_dimension'],
             cs=param['win'])

# train with early stopping on validation set
best_f1 = -numpy.inf
param['clr'] = param['lr']
for e in range(param['nepochs']):

    # shuffle
    shuffle([train_lex, train_ne, train_y], param['seed'])

    param['ce'] = e
    tic = timeit.default_timer()

    # for each sentence in the training compute gradient and update
    for i, (x, y) in enumerate(zip(train_lex, train_y)):
        rnn.train(x, y, param['win'], param['clr'])
        #print ('[learning] epoch '+str(e)+' >> '+str(float("{0:.2f}".format((i+1)* 100. / nsentences))))
        #print ('completed in %.2f (sec) <<\r' % (timeit.default_timer() - tic), file=sys.stdout, flush=True)
       

    # evaluation // back into the real world : idx -> words
    predictions_test = [map(lambda x: idx2label[x],
                        rnn.classify(numpy.asarray(
                        contextwin(x, param['win'])).astype('int32')))
                        for x in test_lex]
    predictions_valid = [map(lambda x: idx2label[x],
                         rnn.classify(numpy.asarray(
                         contextwin(x, param['win'])).astype('int32')))
                         for x in valid_lex]

    # evaluation // compute the accuracy using conlleval.pl
    res_test = conlleval(predictions_test,
                         groundtruth_test,
                         words_test,
                         folder + '/current.test.txt')
    res_valid = conlleval(predictions_valid,
                          groundtruth_valid,
                          words_valid,
                          folder + '/current.valid.txt')

    if res_valid['f1'] > best_f1:

        if param['savemodel']:
            rnn.save(folder)

        best_rnn = copy.deepcopy(rnn)
        best_f1 = res_valid['f1']

        if param['verbose']:
            print('NEW BEST: epoch', e,
                  'valid F1', res_valid['f1'],
                  'best test F1', res_test['f1'], flush=True)

        param['vf1'], param['tf1'] = res_valid['f1'], res_test['f1']
        param['vp'], param['tp'] = res_valid['p'], res_test['p']
        param['vr'], param['tr'] = res_valid['r'], res_test['r']
        param['be'] = e

        subprocess.call(['mv', folder + '/current.test.txt',
                        folder + '/best.test.txt'])
        subprocess.call(['mv', folder + '/current.valid.txt',
                        folder + '/best.valid.txt'])
    else:
        if param['verbose']:
            print('')

    # learning rate decay if no improvement in 10 epochs
    if param['decay'] and abs(param['be']-param['ce']) >= 10:
        param['clr'] *= 0.5
        rnn = best_rnn

    if param['clr'] < 1e-5:
        break

print('BEST RESULT: epoch', param['be'],
      'valid F1', param['vf1'],
      'best test F1', param['tf1'],
      'with the model', folder)

NEW BEST: epoch 0 valid F1 52.89 best test F1 47.69
NEW BEST: epoch 1 valid F1 59.61 best test F1 55.28
NEW BEST: epoch 2 valid F1 66.59 best test F1 64.78
NEW BEST: epoch 3 valid F1 72.83 best test F1 71.87
NEW BEST: epoch 4 valid F1 75.66 best test F1 74.15
NEW BEST: epoch 5 valid F1 77.57 best test F1 75.58
NEW BEST: epoch 6 valid F1 79.47 best test F1 77.34
NEW BEST: epoch 7 valid F1 81.03 best test F1 78.19
NEW BEST: epoch 8 valid F1 81.68 best test F1 78.99
NEW BEST: epoch 9 valid F1 81.88 best test F1 80.74
NEW BEST: epoch 10 valid F1 83.1 best test F1 80.83
NEW BEST: epoch 11 valid F1 83.45 best test F1 80.76
NEW BEST: epoch 12 valid F1 84.33 best test F1 82.4
NEW BEST: epoch 13 valid F1 84.79 best test F1 82.87
NEW BEST: epoch 14 valid F1 85.82 best test F1 83.43
NEW BEST: epoch 15 valid F1 86.22 best test F1 83.84

NEW BEST: epoch 17 valid F1 87.05 best test F1 84.99
NEW BEST: epoch 18 valid F1 87.8 best test F1 85.77


NEW BEST: epoch 21 valid F1 87.85 best test F1 86.22
NEW

/usr/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  return f(*args, **kwds)


### Now we have a look at the embedding of the words in the vector space

In [16]:
print(rnn.emb.eval().shape)
# make sure the row-wise norm is equal to 1
assert(np.sum(np.apply_along_axis(np.linalg.norm, 1, rnn.emb.eval())) == rnn.emb.eval().shape[0]),"Embedding not properly normalized: Each row has to have norm 1."

(573, 3)


### Now we compute check the nearest neighbors for all elements for sanity check

In [17]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(rnn.emb.eval())
distances, indices = nbrs.kneighbors(rnn.emb.eval())

In [18]:
# What are the different label categories?
idx2word

{0: "'d",
 1: "'ll",
 2: "'m",
 3: "'re",
 4: "'s",
 5: "'t",
 6: '72s',
 7: '<UNK>',
 8: 'DIGIT',
 9: 'DIGITDIGIT',
 10: 'DIGITDIGITDIGIT',
 11: 'DIGITDIGITDIGITDIGIT',
 12: 'DIGITDIGITDIGITDIGITDIGITDIGIT',
 13: 'a',
 14: 'abbreviation',
 15: 'abbreviations',
 16: 'about',
 17: 'ac',
 18: 'actually',
 19: 'after',
 20: 'afternoon',
 21: 'again',
 22: 'air',
 23: 'aircraft',
 24: 'airfare',
 25: 'airline',
 26: 'airlines',
 27: 'airplane',
 28: 'airplanes',
 29: 'airport',
 30: 'airports',
 31: 'alaska',
 32: 'all',
 33: 'along',
 34: 'also',
 35: 'am',
 36: 'america',
 37: 'american',
 38: 'amount',
 39: 'an',
 40: 'and',
 41: 'angeles',
 42: 'another',
 43: 'any',
 44: 'anywhere',
 45: 'ap',
 46: 'ap57',
 47: 'ap80',
 48: 'approximately',
 49: 'april',
 50: 'are',
 51: 'area',
 52: 'arizona',
 53: 'around',
 54: 'arrange',
 55: 'arrangements',
 56: 'arrival',
 57: 'arrivals',
 58: 'arrive',
 59: 'arrives',
 60: 'arriving',
 61: 'as',
 62: 'at',
 63: 'atl',
 64: 'atlanta',
 65: 'augu

In [19]:
# show the nearest neighbor to the word 'philadelphia'
idx = 376
[idx2word[x] for x in indices[idx]]

['philadelphia',
 'baltimore',
 'chicago',
 'oakland',
 'boston',
 'pittsburgh',
 'cincinnati',
 'memphis',
 'miami',
 'eighth']

In [23]:
# show the nearest neighbor to the word 'saturday'
idx = 420
[idx2word[x] for x in indices[idx]]

['schedules',
 'arizona',
 'train',
 'taxi',
 'time',
 'highest',
 'schedule',
 'under',
 'times',
 'economy']

In [20]:
# show the nearest neighbor to the word 'august'
idx = 65
[idx2word[x] for x in indices[idx]]

['august',
 'june',
 'november',
 'july',
 'ap',
 'september',
 'december',
 'april',
 'january',
 'rental']

In [21]:
# show the nearest neighbor to the word 'breakfast'
idx = 80
[idx2word[x] for x in indices[idx]]

['breakfast',
 'dinner',
 'sixteen',
 'florida',
 'california',
 'qw',
 'carolina',
 'f',
 'thirteenth',
 'expensive']